In [2]:
import sys
import os
import datetime
import numpy as np
import scipy as sp
import scipy.fftpack
import scipy.io
from scipy.fft import fft, ifft
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
import datetime
%matplotlib inline
import matplotlib as mpl
import matplotlib.ft2font as ft
import matplotlib.font_manager as font_manager
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.labelsize'] = 10

plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 10
mpl.rcParams['xtick.labelsize']  = 8
mpl.rcParams['ytick.labelsize']  = 8
plt.rcParams['figure.dpi'] = 300
import nitime.algorithms as tsa
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import warnings
warnings.filterwarnings('ignore')

font = {'family' : 'arial',
        'weight' : 'normal',
        'size'   : 16}

plt.rc('font', **font)
plt.rc({'axes.labelsize', 'medium'}) 
wellpath = r'C:\Users\Martina\Google Drive\tide_induced_gw_level\spreadsheets\wells_fft'
wpath = os.path.join( wellpath, 'all_mainland2019_m.csv') 
#wpath = os.path.join( wellpath, 'fft_2013_g860.csv') 
#wpath = os.path.join( wellpath, 'fft_2014_f45.csv') 
#wpath = os.path.join( wellpath, 'fft_2015_g3549.csv') 
#wpath = os.path.join( wellpath, 'fft_2017_f319.csv') 
#wpath = os.path.join( wellpath, 'fft_2017_g580a.csv') 
#wpath = os.path.join( wellpath, 'fft_2019_f179.csv')
#wpath = os.path.join( wellpath, 'g3549_2007.csv')
#wpath = r'C:\Users\Martina\PROJECTS\Jacobs model\spreadsheets\hourly\mainland_2010.csv'
df = pd.read_csv(wpath)

FileNotFoundError: [Errno 2] File b'C:\\Users\\Martina\\PROJECTS\\Jacobs model\\tide_induced_gw_level\\spreadsheets\\wells_fft\\all_mainland2019_m.csv' does not exist: b'C:\\Users\\Martina\\PROJECTS\\Jacobs model\\tide_induced_gw_level\\spreadsheets\\wells_fft\\all_mainland2019_m.csv'

In [ ]:
quantity_nan = df['G3549'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['G3549'] = df['G3549'].interpolate(method='linear')
nan_true = df['G3549'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['G3549'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['G3549']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 900
tt =  85

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,1000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('G-3549 Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[708],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['G860'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['G860'] = df['G860'].interpolate(method='linear')
nan_true = df['G860'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['G860'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['G860']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 90
tt =  85

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,100])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('G-860 Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['G580A'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['G580A'] = df['G580A'].interpolate(method='linear')
nan_true = df['G580A'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['G580A'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['G580A']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 140
tt =  135

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,150])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('G-580A Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['F319'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['F319'] = df['F319'].interpolate(method='linear')
nan_true = df['F319'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['F319'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['F319']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 140
tt =  135

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,150])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('F-319 Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['F179'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['F179'] = df['F179'].interpolate(method='linear')
nan_true = df['F179'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['F179'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['F179']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 140
tt =  135

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,150])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('F-179 Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['F45'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['F45'] = df['F45'].interpolate(method='linear')
nan_true = df['F45'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['F45'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['F45']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 380
tt =  360

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,400])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('F-45 Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
df['G852'] = df['G852'].interpolate(method='linear')
nan_true = df['G852'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['G852'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['G852']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 140
tt =  135

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,150])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('G-852 Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['s22t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['s22t'] = df['s22t'].interpolate(method='linear')
nan_true = df['s22t'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s22t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s22t']
tide = df['s22t']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[710])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 16500000
tt =  15000000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,17500000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S22_T Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['s123t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['s123t'] = df['s123t'].interpolate(method='linear')
nan_true = df['s123t'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s123t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s123t']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 16500000
tt =  15000000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,17500000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S123_T Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
from scipy.fft import fft, ifft
quantity_nan = df['s27t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['s27t'] = df['s27t'].interpolate(method='linear')
nan_true = df['s27t'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s27t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s27t']
tide = df['s27t']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 19900000
tt =  18000000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,21000000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S27_T Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['s20gt'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['s20gt'] = df['s20gt'].interpolate(method='linear')
nan_true = df['s20gt'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s20gt'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s20gt']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 12100000
tt =  11000000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,13000000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S20G_T Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
df['s22h'] = df['s22h'].interpolate(method='linear')
nan_true = df['s22h'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s22h'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s22h']
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 335000
tt =  320000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,350000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S22_H Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
df['s123h'] = df['s123h'].interpolate(method='linear')
nan_true = df['s123h'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s123h'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s123h']
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 3000
tt =  2500

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,3500])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S123_H Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
df['s27h'] = df['s27h'].interpolate(method='linear')
nan_true = df['s27h'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s27h'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s27h']
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 1400000
tt =  1300000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,1500000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S27_H Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
df['s20gh'] = df['s20gh'].interpolate(method='linear')
nan_true = df['s20gh'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s20gh'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s20gh']
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 14000
tt =  13000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,15000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S20G_H Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
quantity_nan = df['tide'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

df['tide'] = df['tide'].interpolate(method='linear')
nan_true = df['tide'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['tide'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['tide']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 1780000
tt =  1300000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,1900000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('Tide Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[708],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:
df['s28t'] = df['s28t'].interpolate(method='linear')
nan_true = df['s28t'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s28t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
#tide = df['s28t']
tide = df['s28t']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2

# The next step is to get the frequencies corresponding to the values of the PSD. 
# The fftfreq() utility function does just that. 
# It takes the length of the PSD vector as input as well as the frequency unit. 
# Here, we choose an annual unit: a frequency of 1 corresponds to 1 year (365.25 days),(1 day = 24 hours). 
# We divide 1/(365.25*24) because the original unit is in hours:
fftfreqs_obs = np.fft.fftfreq(len(tide))

# The fftfreq() function returns positive and negative frequencies. 
# We are only interested in positive frequencies here, as we have a real signal:
i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 16500000
tt =  15000000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,17500000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S28_T Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')

In [ ]:

df['s29t'] = df['s29t'].interpolate(method='linear')
nan_true = df['s29t'].isnull().values.any() #check if there are nan values
print('Are there NaN values? ',nan_true)
quantity_nan = df['s29t'].isnull().sum() # how many nan values
print('There are: ', quantity_nan, 'NaN')

#short = df.iloc[0:1242,:]
tide = df['s29t']
tide = df['s29t']*0.3048
coeff_obs = np.polyfit(df['index'],tide, 1)
print('The linear trend coefficients for observed tide are', coeff_obs[0], 'mm hr-1 and ', coeff_obs[1], 'mm')
print('The linear trend coefficients for observed tide are', coeff_obs[0]*24*365.25, 'mm yr-1 and ', coeff_obs[1], 'mm')
trend_obs = np.polyval(coeff_obs, df['index'])
SL_detrend_obs = np.subtract(tide, trend_obs)

tide_fft_obs = np.fft.fft(SL_detrend_obs)

# Once the FFT has been obtained, square  
# its absolute value in order to get the power spectral density (PSD):
tide_psd_obs = np.abs(tide_fft_obs) ** 2
fftfreqs_obs = np.fft.fftfreq(len(tide))

i = fftfreqs_obs > 0
print('Fourier Transform')
print(tide_fft_obs[706])
f2_obs = np.log10(tide_psd_obs[i])
print('power spectral density')
print(tide_psd_obs[706])
print('log10 power spectral density')
print(f2_obs[706])
print('frequencies (hour^-1)')
f1_obs=fftfreqs_obs[i]
print(f1_obs[704:710])
print('fft amplitude (m)')
amp_obs = 2.*np.abs(tide_fft_obs[i])/len(tide_fft_obs)
print(amp_obs[706]) 
#print(2.*np.abs(tide_fft[i])/len(tide_fft))
print('periods (hours)')
print(1/f1_obs[706])
print(len(tide_fft_obs[i]))
print(len(fftfreqs_obs[i]))

tide_fft_selected_obs = tide_fft_obs.copy()
tide_fft_selected_obs[np.abs(fftfreqs_obs) >0.0806] = 0
tide_fft_selected_obs[np.abs(fftfreqs_obs) <0.0804] = 0
t_slow_obs = np.real(sp.fftpack.ifft(tide_fft_selected_obs))
np.savetxt('test.csv',t_slow_obs)

vspan_start1 = fftfreqs_obs[fftfreqs_obs==0.08]
vspan_end1 = 1.92
vspan_start2 = 0.099
vspan_end2 = 0.101
xlim_start = 0.000
xlim_end = 0.10

n2 = 0.076
m2 = 0.079
s2 =0.082
k1 = 0.0405
o1 = 0.035
tt1 = 16500000
tt =  15000000

i = fftfreqs_obs > 0
new_tick_locations = np.array([0.02, 0.04,0.08,0.1])
def tick_function(X):
    V = 1/X
    return ["%.1f" % z for z in V]

plt.figure(figsize=(5,3))

ax = host_subplot(111, axes_class=AA.Axes)
ax.set_xlim([0,xlim_end])
ax.set_ylim([0,17500000])
ax.set_xlabel('Frequency ($\mathregular{hour^{-1}}$)')
ax.set_ylabel('S29_T Spectral Density')
ax.set_title('t (hours)', loc='left')
ax2 = ax.twin() # ax2 is responsible for "top" axis and "right" axis
ax2.axis["top"].major_ticklabels.set_visible(True)
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
plt.stem(fftfreqs_obs[i], tide_psd_obs[i], 'b', \
         markerfmt=" ", basefmt="-b")
ax.axvline(x = fftfreqs_obs[366],linewidth=2,linestyle= '-', color= 'orange', alpha=0.2)
ax.axvline(x = fftfreqs_obs[693],linewidth=2,linestyle= '-', color= 'green', alpha=0.2)
ax.axvline(x = fftfreqs_obs[707],linewidth=2,linestyle= '-', color= 'turquoise', alpha=0.2)
ax.axvline(x = fftfreqs_obs[731],linewidth=2,linestyle= '-', color= 'purple', alpha=0.2)
ax.axvline(x = fftfreqs_obs[340],linewidth=2,linestyle= '-', color= 'red', alpha=0.2)
ax.text(n2, tt, 'N2',color='green', size=8, weight='heavy')
ax.text(m2, tt1, 'M2',color='turquoise', size=8, weight='heavy')
ax.text(s2,tt, 'S2',color='purple', size=8, weight='heavy')
ax.text(k1, tt, 'K1',color='orange', size=8, weight='heavy')
ax.text(o1, tt, 'O1',color='red', size=8, weight='heavy')